In [ ]:
# Пособие по организации микросервисов для безопасной коммуникации с llm 

## Структура

![graphviz.png](attachment:graphviz.png)

## Пример ендпоинтов

|Сервис| Взаимодействие | Пояснение |
|-------|--------------|---------------------------|
|Оркестр|POST /process |Принимает запрос от TG-бота|
|Валидатор|POST /validate|Проверяет безопасность запроса|
|RAG-система|POST /retrieve|Возвращает релевантный контекст|
|LLM-агент|POST /generate|Генерирует ответ с контекстом|
|TG-бот|Webhook → Оркестр|Передаёт сообщения пользователя|

## Часть 1: Средства FastAPI — что, зачем и как применять

|инструмент/фича| что делает | где юзать |
|-|-|-|
|FastAPI()|Создание приложения|Во всех сервисах|
|Pydantic BaseModel|Валидация входных/выходных данных|Все эндпоинты|
|Depends()|Внедрение зависимостей (авторизация, логгеры, конфиги)|Везде, особенно в Оркестре и Валидаторе|
|HTTPException|Возврат ошибок (403, 422, 500 и т.д.)|Валидатор, Оркестр|
|BackgroundTasks|Асинхронные задачи (логирование, уведомления)|Оркестр, LLM-агент|
|CORS Middleware|Разрешить запросы с фронтенда / TG-бота|Если есть UI или публичный API|
|APIRouter|Модульная маршрутизация (разделение эндпоинтов)|Крупные сервисы (Оркестр, RAG)|
|OpenAPI / Swagger UI|Автодокументация API|Все сервисы|
|RateLimiter(через slowapi или redis)|Защита от DDoS / злоупотреблений|Валидатор, Оркестр, TG-бот|
|JWT / OAuth2PasswordBearer|Авторизация пользователей / сервисов|Оркестр, Валидатор|
|Logging + Middleware|Логирование запросов, времени, ошибок|Все сервисы|

### Примеры:

#### Пример: Pydantic модели и эндпоинт в Валидаторе

In [ ]:
# models.py
from pydantic import BaseModel

class ValidationRequest(BaseModel):
    user_id: str
    text: str
    session_id: str = None

class ValidationResult(BaseModel):
    is_allowed: bool
    risk_score: float
    tags: list[str] = []
    reason: str = None

In [ ]:
# main.py
from fastapi import FastAPI, HTTPException
from models import ValidationRequest, ValidationResult

app = FastAPI(title="Валидатор запросов", version="1.0")

@app.post("/validate", response_model=ValidationResult)
async def validate_request(req: ValidationRequest):
    if "запрещённое слово" in req.text.lower():
        raise HTTPException(status_code=403, detail="Toxic content detected")
    return ValidationResult(
        is_allowed=True,
        risk_score=0.1,
        tags=["clean"]
    )

#### Пример: JWT авторизация в Оркестре

In [ ]:
from fastapi import Depends, HTTPException
from fastapi.security import OAuth2PasswordBearer

oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token")

async def verify_token(token: str = Depends(oauth2_scheme)):
    if token != "SECRET_SERVICE_TOKEN":
        raise HTTPException(status_code=401, detail="Invalid token")
    return token

@app.post("/process")
async def process_request(..., token: str = Depends(verify_token)):
    ...

#### Пример: Middleware для логирования

In [ ]:
from fastapi import Request
import time

@app.middleware("http")
async def log_requests(request: Request, call_next):
    start_time = time.time()
    response = await call_next(request)
    duration = time.time() - start_time
    print(f"{request.method} {request.url.path} — {response.status_code} — {duration:.2f}s")
    return response

#### Пример: Использование APIRouter в RAG-сервисе

In [ ]:
# routers/retrieve.py
from fastapi import APIRouter
from models import RetrieveRequest, RetrieveResult

router = APIRouter(prefix="/rag", tags=["RAG"])

@router.post("/retrieve")
async def retrieve_context(req: RetrieveRequest) -> RetrieveResult:
    # ... логика поиска
    return RetrieveResult(context=[...])

In [ ]:
# main.py
from routers.retrieve import router as rag_router

app.include_router(rag_router)

## Часть 2: Docker-контейнеры — примеры как можно оформить

Каждый сервис — отдельный контейнер. Структура одинаковая, меняется только CMD и зависимости.

### Dockerfile (почти универсальный шаблон для всех сервисов)

In [ ]:
# validator/Dockerfile
FROM python:3.11-slim

WORKDIR /app

# Копируем зависимости первыми — для кэширования
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Копируем код
COPY . .

# Создаем непривилегированного пользователя
RUN adduser --disabled-password --gecos '' appuser
USER appuser

# Запускаем сервис на порту 8000
EXPOSE 8000

# Запуск FastAPI через Uvicorn (продакшн-сервер)
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000", "--workers", "2"]

### requirements.txt (пример для Валидатора)

In [ ]:
fastapi==0.110.0
uvicorn==0.29.0
pydantic==2.6.0
python-dotenv==1.0.1
transformers  # если используешь ML для модерации
torch         # опционально

### docker-compose.yml — сборка всей системы (если хочешь потестить локально что-то)

In [ ]:
version: '3.8'

services:
  tg-bot:
    build: ./tg-bot
    ports:
      - "8001:8000"
    environment:
      - TELEGRAM_TOKEN=your_token_here
      - ORCHESTRATOR_URL=http://orchestrator:8000
    depends_on:
      - orchestrator

  orchestrator:
    build: ./orchestrator
    ports:
      - "8002:8000"
    environment:
      - VALIDATOR_URL=http://validator:8000
      - RAG_URL=http://rag:8000
      - LLM_AGENT_URL=http://llm-agent:8000
      - SECRET_TOKEN=supersecret
    depends_on:
      - validator
      - rag
      - llm-agent

  validator:
    build: ./validator
    ports:
      - "8003:8000"

  rag:
    build: ./rag
    ports:
      - "8004:8000"
    environment:
      - QDRANT_HOST=qdrant
      - QDRANT_PORT=6333
    depends_on:
      - qdrant

  llm-agent:
    build: ./llm-agent
    ports:
      - "8005:8000"
    environment:
      - OPENAI_API_KEY=your_key_here

  audit-db:
    image: postgres:15
    environment:
      POSTGRES_DB: audit
      POSTGRES_USER: user
      POSTGRES_PASSWORD: pass
    volumes:
      - audit_data:/var/lib/postgresql/data
    ports:
      - "5432:5432"

volumes:
  audit_data: